# NCPT Paperspace One-click (v0.3)

### Last updated: 25 Dec 2023

Check for updates [here](https://github.com/NoCrypt/paperspace-ncpt)

**!! ONLY RUN IN JUPYTERLAB !!**

![](https://i.ibb.co/M9ZH6qf/image.png)




To hide the code, press on this blue bar on the left of the cell

![](https://i.ibb.co/gFqKVV6/chrome-23-12-25-195649.png)

---


In [ ]:
# fix gdown issue by downgrading it 
!pip uninstall gdown -y
!pip install 'gdown==4.6.0'

#### 1. Configurations

In [ ]:
# Run to initialize and set parameters    . 
import site, sys, time, json, glob, zipfile, re, time, shutil, gdown, subprocess, urllib
from urllib.parse import urlparse, parse_qs, unquote
from typing import List, Tuple, Optional, Any
from ipywidgets import Checkbox, Text, Button, HBox, ToggleButton, Label, Box, Layout, Accordion, VBox, CallbackDispatcher
from IPython.display import display, HTML, clear_output
from pathlib import Path
from subprocess import check_output, check_call
from tempfile import NamedTemporaryFile
from os import getenv, environ

# Mapping resources folders 
paths = {
    "models": "models/Stable-diffusion",
    "loras": "models/Lora",
    "vaes": "models/VAE",
    "embeddings": "embeddings",
    "extensions":  "extensions",  
}

# options for aria2 to use
aria2_opts = [ "--console-log-level=error", "--summary-interval=30", "-j1", "-x16", "-s16", "-k1M", "-c"]


# --- various methods --- #


def filename_wget(url :str, timeout : int=5) -> Optional[str]:

    #batchlinks-downloader method
    req = urllib.request.Request(url, headers={'User-Agent': 'Mozilla/5.0'})
    response = urllib.request.urlopen(req, timeout=timeout)
    forward_url = response.geturl()

    match = re.search(r'filename%3D%22(.+?)%22', forward_url)
    if match:
        filename = urllib.parse.unquote(match.group(1))
        return filename

    #wget method
    dst=Path('/notebooks/.tmp')
    if not dst.exists():
        dst.mkdir()
        
    process = subprocess.Popen(['wget', '--content-disposition', '-P', dst, url], stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
    
    filename = None
    start_time = time.time()

    while True:
        if time.time() - start_time > timeout:
            display("Timeout reached.")
            break
        line = process.stdout.readline()
        if line == '' and process.poll() is not None:
            break
        if 'Saving to:' in line:
            match = re.search(r'Saving to: ‘.+/(.+)’', line)
            if match:
                filename = match.group(1)
                break
    if process.poll() is None:
        process.terminate()
        try:
            process.wait(timeout=0.5)
        except subprocess.TimeoutExpired:
            process.kill()
    if filename:
        file = dst.joinpath(filename)
        if file.exists() :
            file.unlink()
    return filename
    
def delete_symlinks(directory_path:str)  -> None :
    directory = Path(directory_path)
    if not directory.exists() or not directory.is_dir():
        return
    for item in directory.iterdir():  
        if item.is_symlink():
            item.unlink() 
 
# Folder size method 
def get_directory_size(directory_pattern:str) -> str:
    try:
        if '*' in directory_pattern:
            directories = glob.glob(directory_pattern)
        else:
            directory_pattern_path = Path(directory_pattern)
            directories = [directory_pattern] if directory_pattern_path.is_dir() else []
        if not directories:
            return "0B"
        total_size = 0
        for directory in directories:
            output = check_output(['du', '-s', directory], text=True)
            size = int(output.split()[0])  # Size in blocks
            total_size += size * 1024  # Convert to bytes (assuming block size is 1024 bytes)

        # Convert bytes to a human-readable format
        for unit in ['B', 'KB', 'MB', 'GB', 'TB', 'PB']:
            if total_size < 1024:
                return f"{total_size:.2f}{unit}"
            total_size /= 1024

        return f"{total_size:.2f}PB"
    except Exception as e:
        #print("An error occurred:", e)
        return "0B"

# copy dep folder
def copy_recent_files(src_dir:str, dst_dir:str, minutes:int=10) -> None:
    # Calculate the cutoff time
    cutoff_time = time.time() - minutes * 60 *1000
    
    # Ensure the destination directory exists
    dst_dir_path = Path(dst_dir)
    dst_dir_path.mkdir(parents=True, exist_ok=True)

    # Iterate through the immediate files in src_dir
    for item in Path(src_dir).iterdir():
        if item.stat().st_mtime >= cutoff_time:
            if item.is_dir():
                # Use shutil.copytree for directories
                shutil.copytree(
                    item,
                    dst_dir_path / item.name,
                    symlinks=True,
                    ignore=shutil.ignore_patterns("*.pyc"),
                    dirs_exist_ok=True,
                )
            elif item.is_file() and not item.name.endswith('.pyc'):
                # Use shutil.copy2 for files
                shutil.copy2(item, dst_dir_path / item.name)
                
# get current dep folder path
def get_python_package_path()  -> str:
    # Try using site.getsitepackages() first
    package_paths = site.getsitepackages()
    # Add the user site-package directory
    package_paths.append(site.getusersitepackages())

    # Check if these paths exist
    for path_str in package_paths:
        path = Path(path_str)
        if path.exists():
            return str(path)

    # Fallback: Construct the path using sys.executable
    python_executable_dir = Path(sys.executable).parent
    for dir_name in ['site-packages', 'dist-packages']:
        fallback_path = python_executable_dir / dir_name
        if fallback_path.exists():
            return str(fallback_path)
    raise FileNotFoundError("Python package directory not found")


def wget_name(url:str,dst:str) -> Optional[str]:
    # Running wget command to get headers
    output = !wget -S -P {dst} --content-disposition {url}
    # Process the output to extract the filename
    output_str = "\n".join(output)
    # Use regular expression to find the filename
    match = re.search(r'Content-Disposition:.*filename="(.+)"', output_str, re.IGNORECASE)

    if match :
        new_file=f"{dst}/{match.group(1)}"
        #cleaning up the uwanted duplicates if there is any
        !rm -rf {new_file}.*
        return match.group(1)
    return None


class Params:
    # default parameters
    file_path=""
    config_file="config.json"
    controls =[]
    accordions ={}
    links={
        "models": ["https://huggingface.co/NoCrypt/expermental_models/resolve/main/shux7.safetensors"],
        "vaes":["https://huggingface.co/NoCrypt/resources/resolve/main/VAE/any.vae.safetensors"],
        "loras": [],
        "extensions": [],
        "embeddings": [],
        "ui": {"params": {},"enabled": ["https://huggingface.co/NoCrypt/expermental_models/resolve/main/shux7.safetensors","https://huggingface.co/NoCrypt/resources/resolve/main/VAE/any.vae.safetensors"],"keep": [],"names": {},"tabs": []}}

    def __init__(self, control_list:List[str]=[]):
        self.controls = control_list

    def load(self, file:str="links.json")  -> None:
        self.file_path = Path(file)

        if not self.file_path.exists():
            self.save()
        else:
            with self.file_path.open('r') as file:
                self.links = json.load(file)

        for ctrl in self.controls:
            if ctrl.tooltip in self.links["ui"]["params"]:
                ctrl.value = self.links["ui"]["params"][ctrl.tooltip]

    # update parameters from ui 
    def update(self)  -> None:
        for ctrl in self.controls:
            self.links["ui"]["params"][ctrl.tooltip] = ctrl.value
            
    # save parameters to file           
    def save(self) -> None:
        self.update()
        with self.file_path.open('w')  as file:
            json.dump(self.links, file, indent=4)

    # get the status of the ressource enabled/cached
    def get_link_status(self, url:str) -> Tuple[bool, bool]:
        enabled = url in self.links["ui"]["enabled"]
        keep = url in self.links["ui"]["keep"]
        return enabled, keep

    # get the ressource name from parameters
    def name_from_link(self , url:str)-> str:
        if 'names' in self.links["ui"] and url in self.links["ui"]['names']:
            return self.links["ui"]['names'][url]
        else:
            return ""
    def link_from_name(self, name: str) -> str:
        if 'names' in self.links["ui"]:
            for url, n in self.links["ui"]['names'].items():
                if n == name:
                    return url
        return ""
    
    # get the ressource name from parameters
    def set_link_name(self , url:str, name:str) -> None:
        if 'names' not in self.links["ui"] :
            self.links["ui"]['names']={}
        self.links["ui"]['names'][url]=name
        
    # update the ressource status
    def update_link_status(self, url:str, to_enable:bool, to_keep:bool) -> None:
        enabled, keep = self.get_link_status(url)
        if to_enable :
            if not enabled :
                self.links["ui"]["enabled"].append(url)
        else :
            if enabled :
                self.links["ui"]["enabled"].remove(url)
                
        if to_keep :
            if not keep :
                self.links["ui"]["keep"].append(url)
        else :
            if keep :
                self.links["ui"]["keep"].remove(url)
        self.save()

    # adds a new ressource
    def add_link(self, url:str, category:str)-> bool:
        def is_valid_url(url):
            parsed = urlparse(url)
            return bool(parsed.scheme) and bool(parsed.netloc)
            
        url=url.strip()
        if is_valid_url(url) and  url not in self.links[category] : 
            self.links[category].append(url)
            self.links["ui"]["enabled"].append(url)
            
            filename=self.links["ui"]["names"].get(url,'')
            if filename =="" :
                 filename=filename_wget(url)     
            if filename :
                self.links["ui"]["names"][url]=filename if "login" not in filename else f"[login error]{url}"
            self.get_accordion(category)
            self.save()
            return True
        return False

        
        
    # removes the specified ressource
    def remove_link(self, category:str, url:str) -> None:
        if url in self.links[category] :
            self.links[category].remove(url)
        if url in self.links["ui"]["enabled"] :
            self.links["ui"]["enabled"].remove(url)
        if url in self.links["ui"]["keep"] :
            self.links["ui"]["keep"].remove(url)
        if url in self.links["ui"]["names"] :
            del self.links["ui"]["names"][url]
        self.get_accordion(category) 
        self.save()


    # create or update the ressource manager 
    def get_accordion(self, category:str, refresh:bool=False) -> Accordion:
        is_new=False
        if category not in self.accordions:
            is_new=True
            self.accordions[category] = Accordion(layout=Layout(width='52%'))
            def on_accordion_change(change):
                if "tabs" not in self.links["ui"] :
                    self.links["ui"]["tabs"]=[]
                
                if change['name'] == 'selected_index' and change['new'] is not None:
                    if category not in self.links["ui"]["tabs"]:
                        self.links["ui"]["tabs"].append(category)
                else :
                    if category in self.links["ui"]["tabs"]:
                        self.links["ui"]["tabs"].remove(category)
            self.accordions[category].observe(on_accordion_change, names='selected_index')

        if (len(self.accordions[category].children)-1) !=len(self.links[category]) or refresh or len(self.links[category])==0:
            vbox_children = []
            vbox_children.append(self.create_add_link(category))
            for url in self.links[category]:
                element_name=self.name_from_link(url)
                if element_name == "" :
                    element_name=url
                hbox = HBox(list(self.create_action_buttons(url, category)) + [Label(value=element_name)])
                vbox_children.append(hbox)
  
            self.accordions[category].children = [VBox(vbox_children)]
    
        self.update_accordion_title(category)
        if is_new :
            if "tabs" in self.links["ui"] and self.links["ui"]["tabs"]:
                if category in self.links["ui"]["tabs"] :
                    self.accordions[category].selected_index = 0
                else:
                    self.accordions[category].selected_index = None

        return self.accordions[category]
    
    # update the count of resources 
    def update_accordion_title(self, category : str) -> None:
        # Display the update button at the end
        keep_cpt=0
        enabled_cpt=0
        for url in self.links[category] :
            if url in self.links["ui"]["keep"] :
                keep_cpt=keep_cpt+1 
            if url in self.links["ui"]["enabled"] :
                enabled_cpt=enabled_cpt+1
        self.accordions[category].set_title(0, str(len(self.links[category])) +" "+ category + " ["+str(enabled_cpt)+"/" +str(keep_cpt) +"]")
        
    #---- ui components ----# 
    # create action buttons for the link manager
    def create_action_buttons(self, url :str , category : str) -> Tuple[ToggleButton, ToggleButton, Button]:
        enabled, keep = self.get_link_status(url)
        
        toggle_enable =ToggleButton(
            layout=Layout(width='90px'),
            value=enabled, 
            description="Enable",
            tooltip="Enable this model",
            button_style=('success' if enabled else ''),
        )
    
        toggle_keep =ToggleButton(
            layout=Layout(width='90px'),
            value=keep, 
            description="Cache",
            tooltip="Cache this model into notebooks",
            button_style=('success' if keep else ''),
        )
    
        delete_button = Button(
            tooltip="Delete this model",
            button_style='danger',     
            layout=Layout(width='30px'),
            icon='trash-o' 
        )
        
        def on_toggle(cc:CallbackDispatcher):
            self.update_link_status(url,toggle_enable.value,toggle_keep.value)
            toggle_keep.button_style = 'success' if toggle_keep.value else ''
            toggle_enable.button_style = 'success' if toggle_enable.value else ''
            self.update_accordion_title(category)
            self.save()
        
        def on_delete(bb:Button):
            self.update_link_status(url, False, False)
            self.remove_link(category,url)
            self.save()

        toggle_enable.observe(on_toggle, 'value')
        toggle_keep.observe(on_toggle, 'value')
        delete_button.on_click(on_delete)
        return toggle_enable , toggle_keep , delete_button

    # create add ressource ui
    def create_add_link(self, category:str)-> HBox:
        link_control = Text(
            value="",
            placeholder=category + " link...",
            layout=Layout(width='80%'),
        )
        
        add_button = Button(
            description='Add',
            layout=Layout(width='20%'),
            icon='plus' 
        )
    
        def on_add(bb:Button) -> None:            
            description='Adding ...'
            add_button.button_style="warning"
            add_button.icon="refresh"
            if self.add_link(link_control.value,category ) :
                add_button.button_style=""
                add_button.icon="plus"
                description='Add'
            else :
                add_button.button_style="danger"
                add_button.icon="plus"
                description='Add'
                
        add_button.on_click(on_add)
        return HBox([link_control, add_button])
        
    # creating output zip/clean tools 
    def create_zip_tool(self) -> HBox:
        #view
        zip_button = Button(
            description='Zip',
            tooltip='Zip folder',
            icon='cube' 
        )
        clean_button = Button(
            description='Clean',
            tooltip='Cleaning folder',
            icon='trash-o'
        )
        zip_clean_button = Button(
            description='Zip & Clean',
            button_style='info', 
            tooltip='Zip folder and clean it up',
            icon='codepen'
        )
            
        def on_zip(bb:Button):
            display("Cleaning..a.")
            bb.description = bb.description.split('>')[0]
            is_cleaning = "clean" in bb.description.lower()
            is_zipping = "zip" in bb.description.lower()
            cleanup_dir = Path(output_control.value)
            if not cleanup_dir.exists():
                display("The folder doesn't exist.")
                bb.button_style = 'danger'  
                bb.icon="times"
                
            if any(cleanup_dir.iterdir()) :
                if is_zipping: 
                    zipped_name = f"{cleanup_dir.name}_{time.strftime('%Y-%m-%d_%H%M')}.zip" 
                    !zip -r {zipped_name} {cleanup_dir}
                    display(f"{zipped_name} zipped!")
                if is_cleaning :
                    folder_name=cleanup_dir.name
                    !rm -rf {cleanup_dir}
                    cleanup_dir.mkdir(exist_ok=True)
                    display(f"{folder_name} cleaned!")
                bb.button_style = 'success' 
                bb.icon="check"
                
            else :
                display("Empty folder.")
                bb.button_style = 'warning' 
        
        #listeners
        zip_button.on_click(on_zip)
        #clean_button.on_click(on_zip)
        zip_clean_button.on_click(on_zip)
    
        #return ui
        return HBox([
            output_control, 
            zip_button, 
            #clean_button,
            zip_clean_button
        ])
           
   
    #---- views ----#
    # ressources manager
    def view_links(self) -> None:
        for category, urls in params.links.items():
            if category != "ui":
                display(HBox([Box(layout=Layout(width='18%')), self.get_accordion(category) ]))

    # output manager 
    def show_zip_tools(self)-> None:
        display(self.create_zip_tool())


    
    @property 
    def options(self) -> Any:
        return self.links["ui"]["params"] 

    @property
    def commandline_arguments(self) -> str:
        arg_list = ["--enable-insecure-extension-access --disable-safe-unpickle --theme dark  --listen --port 6006"]
        if self.options["hf_token"] != "":
            arg_list.extend(["--hf-token-out", self.hf_token])
        if self.options["update_extensions"]  is True:
            arg_list.append("--update-all-extensions")
        if self.options["save_config"]  is True and self.options["config_file"] != "":
            arg_list.extend(["--ui-settings-file", self.options["config_file"]])
        if self.options["save_tasks"]  is True :
            arg_list.extend(["--agent-scheduler-sqlite-file", "/notebooks/cached/tasks.sqlite3"])
        if self.options["extra_cmd"] != "":
            arg_list.extend([x for x in self.options["extra_cmd"].split(" ") if x != ""])
        return " ".join(arg_list)

# css
style = {"description_width": "25%"}
layout = {"width": "70%"}

label_style = HTML(
    "<style> .widget-label, .widget-label-basic >  span { color: white !important; font-weight: bold;}  </style>"
)

# checkboxes
cached_config_control = Checkbox(
    value=False,  # Keep it false as default, presistent is good but I might change config in the future, also it's much more idiot-proof this way
    description="Persistent config",
    layout=layout,
    style=style,
    tooltip='save_config',
)

cached_tasks_control = Checkbox(
    value=False,  
    description="Persistent task scheduler",
    layout=layout,
    style=style,
    tooltip='save_tasks',
)

persistent_output_control = Checkbox(
    value=False,  # Keep it false as default, I just prefer it off by default like in old colab
    description="Save outputs to notebooks",
    layout=layout,
    style=style,
    tooltip='save_output',
)

dep_size=get_directory_size("/storage/a1111_dependencies")
force_cache_control = Checkbox(
    value=False,  # Keep it false as default, cuz why?!
    description= f"Force re-cache dependencies [{dep_size}]",
    layout=layout,
    style=style,
    tooltip='force_cached',
)
trash_size=get_directory_size('/notebooks/.Trash*')  
clean_trash_control = Checkbox(
    value=False,  
    description=f"Clean trash folder [{trash_size}]",
    layout=layout,
    style=style,
    tooltip='clean_trash',
)

cache_size=get_directory_size('/notebooks/cached')
clean_cached_control = Checkbox(
    value=True,  
    description=f"Force re-cache resources [{cache_size}] (removes tasks too)",
    layout=layout,
    style=style,
    tooltip='clean_cached',
)

extn_update_control = Checkbox(
    value=True, # You would want newer extensions by default
    description="Update all extensions on launch",
    layout=layout,
    style=style,
    tooltip='update_extensions',
)

# textboxes
hf_token = Text(
    value="",
    description="HuggingFace token (optional)",
    placeholder="hf_xyz",
    style=style,
    layout=layout,
    tooltip='hf_token',
)
commandline_arg_control = Text(
    value="--xformers",
    description="Extra commandline arguments",
    style=style,
    layout=layout,
    tooltip='extra_cmd',
)
output_control = Text(
    value="/notebooks/outputs/",
    description="Outputs path",
    placeholder="/notebooks/outputs/",
    style= {"description_width": "46%"},
    layout=Layout(width='38%'),
    tooltip='zip_folder',
)

# params instance with all the controls to save
params=Params([cached_config_control,cached_tasks_control,persistent_output_control,force_cache_control,clean_cached_control,clean_trash_control,extn_update_control,output_control,hf_token,commandline_arg_control])
params.load('links.json')

# textboxes
display(
    hf_token,
    commandline_arg_control,
)
params.show_zip_tools(),

# checkboxes
display(
    cached_config_control,
    cached_tasks_control,
    persistent_output_control,
    force_cache_control,
    clean_cached_control,
    clean_trash_control,
    extn_update_control,
)

# show ressource manger
params.view_links()

### 2. Start 🚀

In [ ]:
# Run this when you're ready    . 

# Can't run without params
class StopExecution(Exception):
    def _render_traceback_(self):
        pass
try:
    params
except NameError:
    display("Please run the configurations block")
    raise StopExecution

# showing the future link
display(f"Will be ready in a few at https://tensorboard-{getenv('PAPERSPACE_FQDN')} ...")

# read controllers and save parameters
params.update()
params.save()

#cleaning trash
if params.options["clean_cached"]:
    display("Cleaning cashed files... " )
    !rm -rf '/notebooks/cached' 
                
#cleaning trash
if params.options["clean_trash"]:
    trash_size=get_directory_size('/notebooks/.Trash*')
    if trash_size!="0G" and  trash_size!="0B" :
        display("Cleaning trash files... " + trash_size)
        directories = glob.glob('/notebooks/.Trash*')
        for directory in directories:
            try:
                check_output(['rm', '-rf', directory])
                display(f"Deleted: {directory}")  
            except CalledProcessError as e:
                display(f"Error deleting {directory}: {e}")    
            
# paperspace-provided directories
notebook_dir = Path("/notebooks")
storage_dir = Path("/storage")

# local-only semi-persistent storage
content_dir = Path("/content")

# # storage directory
# storage_symlink = notebook_dir.joinpath("storage")
# if not all(
#     (
#         storage_symlink.is_symlink(),
#         storage_symlink.resolve() == storage_dir,
#     )
# ):
#     # recreate the symlink
#     storage_symlink.unlink(missing_ok=True)
#     storage_symlink.symlink_to(storage_dir, target_is_directory=True)


# symlink the directory for easier access
content_dir.mkdir(exist_ok=True)
content_symlink = notebook_dir.joinpath("content")
if not all(
    (
        content_symlink.is_symlink(),
        content_symlink.resolve() == content_dir,
    )
):
    # recreate the symlink
    content_symlink.unlink(missing_ok=True)
    content_symlink.symlink_to(content_dir, target_is_directory=True)

# where we put deps
dep_cache_dir = storage_dir.joinpath("a1111_dependencies")

# where models will go
sdw_dir = content_dir.joinpath("sdw")
models_dir = sdw_dir.joinpath("models")
extensions_dir = sdw_dir.joinpath("extensions")
output_dir = notebook_dir.joinpath("outputs/") 
cached_dir = notebook_dir.joinpath("cached")
if not cached_dir.exists():
    cached_dir.mkdir(exist_ok=True)
    
# download repo if needed
config_json = content_dir.joinpath("sdw/config.json")
cached_config_file = notebook_dir.joinpath("config.json")

if not config_json.exists() :
    # repo isn't here so LET'S GOOOOOOOOOO
    aria2_task = "\n".join(
        [
            "https://huggingface.co/NoCrypt/fast-repo/resolve/main/repo.tar.lz4",
            "\tout=repo.tar.lz4",
            "https://huggingface.co/NoCrypt/fast-repo/resolve/main/cache.tar.lz4",
            "\tout=cache.tar.lz4",
        ]
    )

    with NamedTemporaryFile("w") as aria2_file:
        aria2_file.write(aria2_task)
        aria2_file.flush()
        display("Downloading repo and cache. This may take a while...")
        check_call(
            ["aria2c", f"--input-file={aria2_file.name}"] + aria2_opts,
            cwd=content_dir,
        )

    display("Extracting repo...")
    check_call(["tar", "-xI", "lz4", "-f", "repo.tar.lz4", "--directory=/"], cwd=content_dir)
    content_dir.joinpath("repo.tar.lz4").unlink()

    display("Extracting cache...")
    check_call(["tar", "-xI", "lz4", "-f", "cache.tar.lz4", "--directory=/"], cwd=content_dir)
    content_dir.joinpath("cache.tar.lz4").unlink()


# deal with persistent config 
if params.options["save_config"] :
    if not cached_config_file.exists():
        !cp {config_json} {notebook_dir}
    config_json=cached_config_file
params.options["config_file"]= str(config_json)

# deal with persistent output 
if params.options["save_output"]:
    if not output_dir.exists():
        output_dir.mkdir(exist_ok=True)
    !sed -i 's@"outdir_txt2img_samples": "outputs/txt2img-images"@"outdir_txt2img_samples": "{output_dir}/txt2img-images"@' {config_json}
    !sed -i 's@"outdir_img2img_samples": "outputs/img2img-images"@"outdir_img2img_samples": "{output_dir}/img2img-images"@' {config_json}
    !sed -i 's@"outdir_extras_samples": "outputs/extras-images"@"outdir_extras_samples": "{output_dir}/extras-images"@' {config_json}
    !sed -i 's@"outdir_txt2img_grids": "outputs/txt2img-grids"@"outdir_txt2img_grids": "{output_dir}/txt2img-grids"@' {config_json}
    !sed -i 's@"outdir_img2img_grids": "outputs/img2img-grids"@"outdir_img2img_grids": "{output_dir}/img2img-grids"@' {config_json}
    !sed -i 's@"outdir_save": "log/images"@"outdir_save": "{output_dir}/log/images"@' {config_json}

# remove unwanted extensions from nocrypt packaging
pixelization_dir = extensions_dir.joinpath("pixelization")
photope_dir = extensions_dir.joinpath("sd-photopea-embed-lazy")
!rm -rf {pixelization_dir}
!rm -rf {photope_dir}
    
#Adds wildcards
wildcards_dir_src = notebook_dir.joinpath("wildcards")
wildcards_dir_dest = extensions_dir.joinpath("Umi-AI-debloat/")
if not wildcards_dir_src.exists():
    wildcards_dir_src.mkdir(exist_ok=True)
if wildcards_dir_dest.exists():
    !cp -r {wildcards_dir_src} {wildcards_dir_dest}

# flush dependency cache if forced
if params.options["force_cached"] :
    if dep_cache_dir.exists() :
        display("Forced removal of dependencies...")
        shutil.rmtree(dep_cache_dir)

current_dep_dir=Path(get_python_package_path())

# set up dependency cache
if dep_cache_dir.exists() :
    if current_dep_dir.exists():
        display("Copying dependencies from cache...")

        # recursively copy back files from the cached folder to the current one 
        shutil.copytree(
            dep_cache_dir,
            current_dep_dir,
            symlinks=True,
            dirs_exist_ok=True,
        )
else :
    dep_cache_dir.mkdir(exist_ok=True)
    start_time = time.time()
    check_call(
        ["bash", "./webui.sh", "-f", params.commandline_arguments, "--exit"],
        cwd=content_dir.joinpath("sdw"),
    )
    end_time = time.time()
    # updated no need to update later on
    params.options["update_extensions"]=False
    copy_recent_files(current_dep_dir, dep_cache_dir, round((end_time - start_time) / 60)+1)

# func to download resources
def acquire_checkpoints(checkpoint_urls: list[str], fldr: str = "Stable-diffusion", is_cached: bool = False) -> None:

    sdw_fldr=paths.get(fldr, fldr)
    dst=sdw_dir.joinpath(sdw_fldr)
    real_dst=sdw_dir.joinpath(sdw_fldr)

    # cleaning symlinks and cache uncached
    if(is_cached) :
        #cleaning symbolic links
        sym_dst=sdw_dir.joinpath(paths.get(category, category))
        if sym_dst.exists() :
            delete_symlinks(sym_dst)
        #cleaning cached
        dst=cached_dir.joinpath(fldr)
        if not dst.exists():
            dst.mkdir(exist_ok=True)
        else :
            for item in dst.iterdir(): 
                if item.name.startswith(".") :
                    continue
                url=params.link_from_name(item.name)
                enabled, keep=params.get_link_status(url)
                if not keep and not enabled:
                    #delete from cache
                    item.unlink()
                    
                elif not keep and enabled:
                    # move away from cache
                    moved_dst=real_dst.joinpath(item.name)
                    if not moved_dst.exists() :
                         shutil.move(item, real_dst)
                    else :
                        item.unlink()               
        if real_dst.exists():
            for item in real_dst.iterdir(): 
                if item.name.startswith(".") :
                    continue
                url=params.link_from_name(item.name)
                enabled, keep=params.get_link_status(url)
                if  keep and enabled:
                    # move away from cache
                    moved_dst=dst.joinpath(item.name)
                    if not moved_dst.exists() :
                        shutil.move(item,dst)
                    else :
                        item.unlink()        
                                  
    if len(checkpoint_urls) == 0 :
        return  
    aria2_task_lines = []
    for url in checkpoint_urls:
        filename=params.name_from_link(url)
        if fldr=="extensions" :
            if filename == "" :
                filename=urlparse(url).path.split('/')[-1]
                filename=filename.replace('.git','')
            if not dst.joinpath(filename).exists():
                !git clone --quiet {url} {dst}/{filename} 
            else:
                !cd {dst}/{filename} && git fetch --quiet > /dev/null 2>&1 && git merge --quiet
        if 'drive.google' in url:
            if not dst.joinpath(filename).exists():
                downloaded_file_path = gdown.download(url, str(dst)+"/", quiet=True, fuzzy=True) 
                filename = Path(downloaded_file_path).name
                params.set_link_name(url,filename)
        elif filename=="" :
            filename= wget_name(url,dst)
            if filename :
                params.set_link_name(url,filename)
        elif "login" not in filename:
            if not dst.joinpath(filename).exists():
                aria2_task_lines.extend([url,f"\tout={filename}"])
            params.set_link_name(url,filename)
          
    if len(aria2_task_lines) > 0:
        with NamedTemporaryFile("w") as aria2_file:
            aria2_file.write("\n".join(aria2_task_lines))
            aria2_file.flush()
            check_call(["aria2c", f"--input-file={aria2_file.name}"] + aria2_opts,cwd=dst,)
            
    if is_cached :
        for file in dst.iterdir():
            # hidden files
            if file.name.startswith('.') or file.name.endswith('.aria2') :  
                continue

            #extensions
            if fldr =="extensions":
                extension_symlink = Path(real_dst / file.name) 
                if not all(
                    (
                        extension_symlink.is_symlink(),
                        extension_symlink.resolve() == file,
                    )
                ):
                    # recreate the symlink
                    extension_symlink.unlink(missing_ok=True)
                    extension_symlink.symlink_to(file, target_is_directory=True)
                continue

            #other resources
            original_file = real_dst / file.name
            symlink_path = file
            if original_file.exists() or original_file.is_symlink():
                #display(f"Removing existing file or symlink: {original_file}")
                original_file.unlink()
            try:
                original_file.symlink_to(symlink_path)
                #display(f"Created symbolic link: {original_file} -> {symlink_path}")
            except FileExistsError as e:
                #display(f"Failed to create symbolic link due to existing file: {e}")
                pass
  

# download, cache and link ressources 
for category, urls in params.links.items():
    if category =="ui" :
        continue
  
    active_urls= []
    persistent_urls= []
        
    if len(urls) >0 :
        for url in urls:
            enabled, keep = params.get_link_status(url)
            if enabled :
                if not keep :
                    if url not in active_urls:
                        active_urls.append(url)
                    if url in persistent_urls:
                        persistent_urls.remove(url)
                else :
                    if url not in persistent_urls:
                        persistent_urls.append(url)
                    if url in active_urls:
                        active_urls.remove(url)
            else:
                if url in active_urls:
                        active_urls.remove(url)
                if url in persistent_urls:
                        persistent_urls.remove(url)

    
    display(f"[{category}] downloading... {(len(persistent_urls)+len(active_urls))} resources")
    acquire_checkpoints(persistent_urls, category, True) 
    acquire_checkpoints(active_urls,category)
    
params.save()

# start. the thing
environ["COMMANDLINE_ARGS"] = params.commandline_arguments
environ["REQS_FILE"] = "requirements_versions.txt"

display("Starting 'tensorboard'...")
display(f"Access via this URL: https://tensorboard-{getenv('PAPERSPACE_FQDN')}")

webui_user = f"""#!/bin/bash
export COMMANDLINE_ARGS='{params.commandline_arguments}'
venv_dir='-'
"""
content_dir.joinpath("sdw/webui-user.sh").write_text(webui_user)
check_call(["/usr/bin/env", "bash", "./webui.sh", "-f"], cwd=content_dir.joinpath("sdw"))


In [ ]:
# code to update webui
!cd {sdw_dir} && git pull